# General Imports and Pre-Requisites

In [4]:
import carla
import random
import numpy as np
import cv2
import queue

from carla import Transform, Location
from utils import download_installer, get_actor_blueprints, remove_actors

import settings

settings.init()
download_installer()

# Connect to the client and get the world object
client = carla.Client("localhost", settings.carla_port)
world = client.get_world()
spectator = world.get_spectator()

# Get the blueprint library and the spawn points for the map
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

all_id = []

# Adoption of the World to meet our Visualisation needs

In [ ]:
# List possible worlds
#print(client.get_available_maps())
#client.load_world('Town04') # larger towns with crossing Town03, Town05

In [5]:
# remove trees to have a better view
world.unload_map_layer(carla.MapLayer.Foliage)

# Add Camera

In [6]:
cameras = []
camera_data = {}

def camera_callback(camera):
    state = camera.id

    def process(image):
        if state in camera_data:
            image.save_to_disk('output/' + str(state) + '/' + '%.6d.jpg' % image.frame)
            camera_data[state].put(np.reshape(
                np.copy(image.raw_data), (image.height, image.width, 4)
            ))

    return process

for i in range(settings.number_of_cameras):
    if i >= settings.number_of_cameras:
        break
    else:
        input("Press Enter to continue...")

    spectator_location = world.get_spectator().get_transform()
    camera_bp = bp_lib.find("sensor.camera.rgb")

    camera = world.spawn_actor(camera_bp, spectator_location)
    cameras.append(camera)

    # Get gamera dimensions and initialise dictionary
    image_w = camera_bp.get_attribute("image_size_x").as_int()
    image_h = camera_bp.get_attribute("image_size_y").as_int()
    camera_data[camera.id] = queue.Queue()
    camera_data[camera.id].put(np.zeros((image_h, image_w, 4)))

    # Start camera recording
    callback = camera_callback(camera)
    camera.listen(callback)

# Game loop
while True:
    for camera in cameras:
        if camera.id in camera_data and not camera_data[camera.id].empty():
            cv2.imshow("RGB Camera " + str(camera.id), camera_data[camera.id].get())

    # Quit if user presses 'esc'
    key = cv2.waitKey(1)
    if key == 27:
        break

# Close OpenCV window when finished
cv2.destroyAllWindows()

# Add Vehicle

In [7]:
# Add traffic to the simulation
while world.get_actors().filter("*vehicle*").__len__() < settings.number_of_vehicles:
    vehicle_bp = random.choice(bp_lib.filter("vehicle"))
    npc = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

    if npc is not None:
        print("Vehicle {} spawned".format(npc.id))

# Set the all vehicles in motion using the Traffic Manager
for v in world.get_actors().filter("*vehicle*"):
    v.set_autopilot(True)

Vehicle 27 spawned
Vehicle 28 spawned
Vehicle 29 spawned
Vehicle 30 spawned
Vehicle 31 spawned
Vehicle 32 spawned
Vehicle 33 spawned
Vehicle 34 spawned
Vehicle 35 spawned
Vehicle 36 spawned
Vehicle 37 spawned
Vehicle 38 spawned
Vehicle 39 spawned
Vehicle 40 spawned
Vehicle 41 spawned
Vehicle 42 spawned
Vehicle 43 spawned
Vehicle 44 spawned
Vehicle 45 spawned
Vehicle 46 spawned
Vehicle 47 spawned
Vehicle 48 spawned
Vehicle 49 spawned
Vehicle 50 spawned
Vehicle 51 spawned
Vehicle 52 spawned
Vehicle 53 spawned
Vehicle 54 spawned
Vehicle 55 spawned
Vehicle 56 spawned
Vehicle 57 spawned
Vehicle 58 spawned
Vehicle 59 spawned
Vehicle 60 spawned
Vehicle 61 spawned
Vehicle 62 spawned
Vehicle 63 spawned
Vehicle 64 spawned
Vehicle 65 spawned
Vehicle 66 spawned
Vehicle 67 spawned
Vehicle 68 spawned
Vehicle 69 spawned
Vehicle 70 spawned
Vehicle 71 spawned
Vehicle 72 spawned
Vehicle 73 spawned
Vehicle 74 spawned
Vehicle 75 spawned
Vehicle 76 spawned


# Add Pedestrians

In [8]:
SpawnActor = carla.command.SpawnActor
walkers_list = []
all_id = []

# 2. we spawn the walker object
batch = []
walker_speed = []
for i in range(settings.number_of_walkers):
    spawn_point = random.choice(spawn_points)
    walker_bp = random.choice(get_actor_blueprints(world, "walker.pedestrian.*", "all"))
    # set as not invincible
    if walker_bp.has_attribute("is_invincible"):
        walker_bp.set_attribute("is_invincible", "false")
    # set the max speed
    if walker_bp.has_attribute("speed"):
        if random.random() > settings.percentage_pedestrians_running:
            # walking
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[1])
        else:
            # running
            walker_speed.append(walker_bp.get_attribute("speed").recommended_values[2])
    else:
        print("Walker has no speed")
        walker_speed.append(0.0)
    batch.append(SpawnActor(walker_bp, spawn_point))
results = client.apply_batch_sync(batch, True)
walker_speed2 = []
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list.append({"id": results[i].actor_id})
        walker_speed2.append(walker_speed[i])
walker_speed = walker_speed2

# 3. we spawn the walker controller
batch = []
walker_controller_bp = world.get_blueprint_library().find("controller.ai.walker")
for i in range(len(walkers_list)):
    batch.append(
        SpawnActor(walker_controller_bp, carla.Transform(), walkers_list[i]["id"])
    )
results = client.apply_batch_sync(batch, True)
for i in range(len(results)):
    if results[i].error:
        print(results[i].error)
    else:
        walkers_list[i]["con"] = results[i].actor_id

# 4. we put together the walkers and controllers id to get the objects from their id
for i in range(len(walkers_list)):
    all_id.append(walkers_list[i]["con"])
    all_id.append(walkers_list[i]["id"])
all_actors = world.get_actors(all_id)
world.tick()

# 5. initialize each controller and set target to walk to (list is [controler, actor, controller, actor ...])
# set how many pedestrians can cross the road
world.set_pedestrians_cross_factor(settings.percentage_pedestrians_crossing)
for i in range(0, len(all_id), 2):
    all_actors[i].start()

    # set walk to random point
    all_actors[i].go_to_location(world.get_random_location_from_navigation())
    # max speed
    all_actors[i].set_max_speed(float(walker_speed[int(i / 2)]))

Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position
Spawn failed because of collision at spawn position


# Clear Scene

In [9]:
# Stop the camera when we've recorded enough data
[camera.stop() for camera in cameras]
[remove_actors(world, actor) for actor in ["walker", "vehicle"]]

Removing walker 77
Removing walker 78
Removing walker 79
Removing walker 80
Removing walker 81
Removing walker 82
Removing walker 83
Removing walker 84
Removing walker 85
Removing walker 86
Removing walker 89
Removing walker 90
Removing walker 91
Removing walker 92
Removing walker 93
Removing walker 94
Removing walker 95
Removing walker 96
Removing walker 97
Removing walker 98
Removing walker 99
Removing walker 100
Removing walker 101
Removing walker 102
Removing walker 103
Removing walker 104
Removing walker 105
Removing walker 107
Removing walker 108
Removing walker 109
Removing walker 110
Removing walker 111
Removing walker 112
Removing walker 113
Removing walker 114
Removing walker 115
Removing walker 116
Removing walker 117
Removing walker 118
Removing walker 119
Removing walker 120
Removing walker 121
Removing walker 122
Removing walker 123
Removing walker 124
Removing walker 125
Removing walker 126
Removing walker 127
Removing walker 128
Removing walker 129
Removing walker 130
R

[None, None]